In [1]:
# xmlrpc_server.ipynb

from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler
import xmlrpc.client
import datetime
import pandas as pd
import numpy as np
import pickle


class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

server = SimpleXMLRPCServer(("localhost", 8008),
                            requestHandler=RequestHandler)

server.register_introspection_functions()

stats_server = xmlrpc.client.ServerProxy("http://localhost:8018")

# Добавление в лог через сервер
def add_log(log_line):
    try:
        stats_server.add_log(log_line)
    finally:
        return True

# Тест
def ping():
    add_log('ping')
    return True
server.register_function(ping, 'ping')

# Время сервера
def now():
    add_log('now')
    return datetime.datetime.now()
server.register_function(now, 'now')

# Отображение строкового вида, типа и значений
def show_type(arg):
    add_log('show_type')
    return (str(arg), str(type(arg)), arg)
server.register_function(show_type, 'type')

# Сумма
def test_sum(a, b):
    add_log('test_sum')
    return a + b
server.register_function(test_sum, 'sum')

# Степень
def test_pow(a, b):
    add_log('test_pow')
    return a**b
server.register_function(test_pow, 'pow')

# Проверка нахождения клиента в черном списке c использованием Pandas Data Frame
def black_list_check(sname):
    add_log('black_list_check')
    frame = pd.read_csv('lr1/bad_boys2.csv', header=0, sep=',', encoding='utf8')
    exist = any(frame['Surname'] == sname)
    if (exist == True):
        return sname + ": "+ "bad_boy"
    else:
        return sname + ": "+ "good_boy"

server.register_function(black_list_check, 'black_list_check')

# Проверка нахождения клиента в черном списке по фио и дате рождения c использованием Pandas Data Frame
def black_list_check_by_name_and_birth(surname,name,patronym,birth):
    add_log('black_list_check_by_name_and_birth')
    frame = pd.read_csv('bad_boys2.csv', header=0, sep=',', encoding='utf8')
    # processed_frame = frame.query("Surname == @surname and Name == @name and Patronym = patronym and Birth = birth")
    # exist = len(processed_frame) != 0
    exist = False
    for index, el in frame.iterrows():
        if el['Surname'] == surname and el['Name'] == name and el['Patronym'] == patronym and el['Birth'] == birth:
            exist = True
            break
    return surname + " " + name + " " + patronym + " " + birth+ ": "+ ("bad_boy" if exist else "good_boy")

server.register_function(black_list_check_by_name_and_birth, 'black_list_check_by_name_and_birth')

# Бинарная передача данных
def send_back_binary(bin_data):
    add_log('send_back_binary')
    data = bin_data.data
    return xmlrpc.client.Binary(data)
server.register_function(send_back_binary, 'send_back_binary')

# Инверсия цвета
# На вход изображение RGB размерности (M, N, 3) и (M, N, 1) со значениями 0-255
def send_back_inversion(bin_data):
    add_log('send_back_inversion')
    img_arr = pickle.loads(bin_data.data)
    # img_arr=255-img_arr
    inverse = lambda x: 255 - x

    height = img_arr.shape[0]
    weight = img_arr.shape[1]
    is_monochrome = len(img_arr.shape)==2
    if is_monochrome:
        for i in range(height):
            for j in range(weight):
                img_arr[i][j] = inverse(img_arr[i][j])
    else:
        colors = img_arr.shape[2]
        for i in range(height):
            for j in range(weight):
                for c in range(colors):
                    img_arr[i][j][c]=inverse(img_arr[i][j][c])
    pimg = pickle.dumps(img_arr)
    return xmlrpc.client.Binary(pimg)
server.register_function(send_back_inversion, 'color_inversion')

# Отзеркаливает изображение по вертикали
def rotate_vertical(bin_data):
    add_log('rotate_vertical')
    img_arr = pickle.loads(bin_data.data)
    # img_arr=np.flip(img_arr, axis = 1)
    height = img_arr.shape[0]
    half_weight = img_arr.shape[1]//2
    is_monochrome = len(img_arr.shape)==2
    if is_monochrome:
        for i in range(height):
            for j in range(half_weight):
                img_arr[i][j],img_arr[i][-j-1] = img_arr[i][-j-1], img_arr[i][j]
    else:
        colors = img_arr.shape[2]
        for i in range(height):
            for j in range(half_weight):
                for c in range(colors):
                    img_arr[i][j][c],img_arr[i][-j-1][c] = img_arr[i][-j-1][c], img_arr[i][j][c]

    pimg = pickle.dumps(img_arr)
    return xmlrpc.client.Binary(pimg)
server.register_function(rotate_vertical, 'rotate_vertical')

# Бинаризация по порогу
def threshold_img(bin_data,limit):
    add_log('threshold_img')
    img_arr = pickle.loads(bin_data.data)
    # img_arr = (img_arr > limit)*255
    threshold_val_fun = lambda x: 255 if x>limit else 0

    height = img_arr.shape[0]
    weight = img_arr.shape[1]
    is_monochrome = len(img_arr.shape)==2
    if is_monochrome:
        for i in range(height):
            for j in range(weight):
                img_arr[i][j] = threshold_val_fun(img_arr[i][j])
    else:
        colors = img_arr.shape[2]
        for i in range(height):
            for j in range(weight):
                for c in range(colors):
                    img_arr[i][j][c]=threshold_val_fun(img_arr[i][j][c])

    pimg = pickle.dumps(img_arr)
    return xmlrpc.client.Binary(pimg)
server.register_function(threshold_img, 'threshold_img')

def get_info_from_log(par_type= '',par_start_time= '',par_end_time= ''):
    try:
        return stats_server.get_info_from_log(par_type,par_start_time,par_end_time)
    except:
        return 'statistics server is not working'
server.register_function(get_info_from_log, 'get_info_from_log')


print("Listening on port 8008...")
server.serve_forever()

Listening on port 8008...


127.0.0.1 - - [10/Oct/2023 13:37:01] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2023 13:37:07] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2023 13:37:13] "POST /RPC2 HTTP/1.1" 200 -

KeyboardInterrupt

